<font size=6>HIE Data Cleaning Code<br>Lauren T. Simmonds<br>lauren.t.simmonds@gmail.com<br>December 7, 2021

In [1]:
import pandas as pd
import numpy as np

import boto3
from sagemaker import get_execution_role

from time import sleep
import sys

In [2]:
role = get_execution_role()
s3 = boto3.client("s3")
s3_resource = boto3.resource("s3")

bucket='hie-lts2429'
data_key = '_SQL_query_combined.csv'
data_location = 's3://{}/{}'.format(bucket, data_key)

data = pd.read_csv(data_location)
#data = pd.read_csv("_SQL_query_combined.csv")

C:\Users\Lauren\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
data.shape

(38365875, 26)

In [4]:
pd.options.display.max_columns = None
data.head()

,patient_id,age,gender,race,ethnicity,language,encounter_id,encounter_class,admit_type,facility,admit_source,admit_date,discharge_date,disposition,discharge_notes,reason_for_visit,financial_class,vip_indicator,encounter_category,encounter_type,encounter_description,encounter_code,encounter_code_method,encounter_priority,file_number,tract_fips
0,1,39,M,Unknown,Unknown,en,1072200,OUTPT HOSP,NaN,New Hanover Regional Medical Center,NaN,2014-04-22,NaN,NaN,NaN,NaN,NaN,NaN,Diagnosis,ADMIT,Calculus of ureter,592.1,ICD-9,NaN,1,3.712901e+10
1,1,39,M,Unknown,Unknown,en,1234180,EMERGENCY,Emergency,New Hanover Regional Medical Center,Non-Health Care Facility Point of*,2013-02-14,2013-02-15,A,ROUTINE/HOME,NaN,Blue Cross,NaN,Diagnosis,ADMIT,Sprain of neck,847.0,ICD-9,NaN,1,3.712901e+10
2,1,39,M,Unknown,Unknown,en,1234565,EMERGENCY,Emergency,New Hanover Regional Medical Center,Non-Health Care Facility Point of*,2013-07-10,2013-07-10,A,ROUTINE/HOME,NaN,Blue Cross,NaN,Diagnosis,ADMIT,Open wound(s) (multiple) of unspecified site(s...,879.8,ICD-9,NaN,1,3.712901e+10
3,1,39,M,Unknown,Unknown,en,1234584,EMERGENCY,Emergency,New Hanover Regional Medical Center,Non-Health Care Facility Point of*,2013-07-20,2013-07-20,A,ROUTINE/HOME,NaN,Blue Cross,NaN,Diagnosis,ADMIT,Suture / Staple Removal,105,Free Text,NaN,1,3.712901e+10
4,1,39,M,Unknown,Unknown,en,1235388,HOSP OUTPT SURGERY,Elective,New Hanover Regional Medical Center,PHYSICIAN REFERRAL,2014-04-15,2014-04-15,A,ROUTINE/HOME,NaN,Blue Cross,NaN,Diagnosis,ADMIT,NOT GIVEN,NaN,Free Text,NaN,1,3.712901e+10


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38365875 entries, 0 to 38365874
Data columns (total 26 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   patient_id             int64  
 1   age                    int64  
 2   gender                 object 
 3   race                   object 
 4   ethnicity              object 
 5   language               object 
 6   encounter_id           int64  
 7   encounter_class        object 
 8   admit_type             object 
 9   facility               object 
 10  admit_source           object 
 11  admit_date             object 
 12  discharge_date         object 
 13  disposition            object 
 14  discharge_notes        object 
 15  reason_for_visit       object 
 16  financial_class        object 
 17  vip_indicator          object 
 18  encounter_category     object 
 19  encounter_type         object 
 20  encounter_description  object 
 21  encounter_code         object 
 22  encounter_code_m

In [6]:
# The maximum string length of each column

lengths_full = []
for col in data:
    lengths_full.append((col, data[col].astype(str).str.len().unique().max()))

In [7]:
lengths_full

[('patient_id', 7),
 ('age', 3),
 ('gender', 16),
 ('race', 41),
 ('ethnicity', 26),
 ('language', 7),
 ('encounter_id', 8),
 ('encounter_class', 28),
 ('admit_type', 25),
 ('facility', 38),
 ('admit_source', 45),
 ('admit_date', 10),
 ('discharge_date', 10),
 ('disposition', 92),
 ('discharge_notes', 100),
 ('reason_for_visit', 50),
 ('financial_class', 46),
 ('vip_indicator', 24),
 ('encounter_category', 9),
 ('encounter_type', 7),
 ('encounter_description', 300),
 ('encounter_code', 17),
 ('encounter_code_method', 9),
 ('encounter_priority', 4),
 ('file_number', 2),
 ('tract_fips', 13)]

In [8]:
# All unique values for each column

for col in data:
    print(col, data[col].unique())

patient_id [     1      2      3 ... 599998 599999 600000]
age [ 39  66  47  30  56  55  67  37  68  36  73  49  54  71  62  69  72  76
  64  51  95  61  59  65  86  81  42  34  60 100  78  77  79  70  80  35
  45  84   7  44  19  83  85  58  53  25  27  20  57  50  31  74  90  52
  21  94  38  29  75  22  23  48  18  92  87  46  26  16  17  98  14  63
  82  43  99  40  33  41  10  93  88  91  89  28  15  13   6 103   5  32
   4  12  11  24  96  97   9   8 102 105 101 107 106 120 121 108 109 104
 113 111   3 135   2 110 180 112 117 118 119 127 131 114 128 115 122 137
 124 187 141 123 221 140 139   1 164 142 126 129 130 116 171 125 133 136
 147 144 138 148 146 132 162   0 151 -57 -39]
gender ['M' 'F' 'Female' 'Male' 'UNK' 'MALE' 'UN' 'Undifferentiated' 'FEMALE'
 'UNKNOWN' 'Unknown' 'N']
race ['Unknown' 'Asian' 'White' nan 'American Indian Or Alaska Native'
 'White or Caucasian' 'Black Or African American'
 'Black or African American' 'American Indian or Alaska Native'
 'BI-RACIAL' 'Othe

financial_class [nan 'Blue Cross' 'Other' 'Commercial' 'Medicare' "Veteran's Affairs"
 'Medicaid' 'SELF' 'Tricare/Champus' "Workers' Compensation" 'Self-pay'
 'BCBS' 'O' 'State Health Plan' 'TCE' 'Institutional' 'Self-Pay' 'CARE'
 'WC' 'MCRE ADV' 'Hospice' 'COMM' 'HUMANA MEDICARE OP'
 'Medicare Health Maintenance Organization (HMO)' 'Medicare HMO'
 'Veterans Affairs' 'Liability' "Worker's Compensation" 'PRIVATE PAY'
 'UNITED HEALTHCARE OP' 'Blue Cross-Out of State' 'AARP OP'
 'UNITED HEALTHCARE MEDICARE' 'CAID' 'BLUE MEDICARE OP'
 'AETNA US HEALTHCARE OP' 'TPL OP' 'LIABILITY' 'CB' 'C50' 'C32' 'C'
 'AETNA MEDICARE OPEN OP' 'V' 'SBE' 'GBR' 'CIGNA OP' 'PL' 'BBD'
 'BLUE MEDICARE IP' 'MBX' 'HUMANA OP' 'MBP'
 'UNITED HEALTHCARE MEDICARE I/P' 'GBV' 'CBY' 'CBI'
 'UNITED MEDICAL RESOURCES OP' 'MBK' "Veterans' Affairs" 'XBN' 'XBU'
 'NC HEALTH CHOICE' 'MEDCOST OP' 'AETNA US HEALTHCARE IP' 'PNH'
 'BRUNS CO HD/BCCCP' 'CT' 'HUMANA MEDICARE IP' 'Unknown' 'OT STATE' 'C34'
 'CB1' 'MCD MGD CARE' 'HOSP' 

In [9]:
# Change column to datetime
data["admit_date_mod"] = pd.to_datetime(data.admit_date, infer_datetime_format=True)

In [10]:
# Examime ages

age = data.age.sort_values().unique()
age

array([-57, -39,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,
        11,  12,  13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,
        24,  25,  26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,
        37,  38,  39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,
        50,  51,  52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,
        63,  64,  65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,
        76,  77,  78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,
        89,  90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101,
       102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114,
       115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127,
       128, 129, 130, 131, 132, 133, 135, 136, 137, 138, 139, 140, 141,
       142, 144, 146, 147, 148, 151, 162, 164, 171, 180, 187, 221],
      dtype=int64)

<font size=7>Creation of cleaned columns for categorical features

<font size=7>Gender

In [11]:
data.gender.nunique()

12

In [12]:
data["gender_mod"] = np.nan

In [13]:
data.loc[(data.gender == "Female") | (data.gender == "FEMALE") | (data.gender == "F"), "gender_mod"] = "FEMALE"

data.loc[(data.gender == "Male") | (data.gender == "MALE") | (data.gender == "M"), "gender_mod"] = "MALE"

data.loc[(data.gender == "UNK") | (data.gender == "UN") | (data.gender == "N") | (data.gender == "UNKNOWN") | 
         (data.gender == "Unknown") | (data.gender == "Undifferentiated"), "gender_mod"] = "UNKNOWN"

In [14]:
data.gender_mod.unique()

array(['MALE', 'FEMALE', 'UNKNOWN'], dtype=object)

<font size=7>Race

In [15]:
data.race.nunique()

66

In [16]:
data["race_mod"] = np.nan

In [17]:
data.loc[(data.race == "White") | (data.race == "WHITE") | (data.race == "White or Caucasian") | 
              (data.race == "Caucasian or White") | (data.race == "Caucasian") |
              (data.race == "CAUCASIAN") | (data.race == "5^CAUCASIAN^99TRA") |
              (data.race == "W^WHITE^99TRA") | (data.race == "European"), "race_mod"] = "WHITE"

data.loc[(data.race == "Asian") | (data.race == "ASIAN") | (data.race == "A^ASIAN^99TRA") | 
              (data.race == "2^ASIAN^99TRA"), "race_mod"] = "ASIAN"

data.loc[(data.race == "Black Or African American") | (data.race == "Black or African American") | 
              (data.race == "BLACK") | (data.race == "AFRICAN AM") | (data.race == "AFRICAN AMERICAN") |
              (data.race == "African American") | (data.race == "Black") |
              (data.race == "3^BLACK^99TRA") | (data.race == "B^BLACK^99TRA"), "race_mod"] = "BLACK"

data.loc[(data.race == "Unknown") | (data.race == "UNKNOWN") | (data.race == "U^UNKNOWN^99TRA") | 
              (data.race == "UNAVAILABLE/DECLINED") | (data.race == "Unavailable or Patient decline") |
              (data.race == "PATIENT REFUSED") | (data.race == "PT DECLINE") | 
              (data.race.isnull()), "race_mod"] = "UNKNOWN"

data.loc[(data.race == "Other Race") | (data.race == "OTHER") | (data.race == "OTHER RACE") | 
              (data.race == "6^OTHER RACE^99TRA") | (data.race == "BI-RACIAL"), "race_mod"] = "OTHER"

data.loc[(data.race == "American Indian Or Alaska Native") | (data.race == "American Indian or Alaska Native") | 
              (data.race == "American Indian or Alaskan Native") | (data.race == "American Indian") | 
              (data.race == "American Indian or Native American") | (data.race == "1^AMERCA IND^99TRA") | 
              (data.race == "Alaska Native") | (data.race == "AMERICAN INDIAN") | (data.race == "AMERCA IND") | 
              (data.race == "INDIAN") | (data.race == "I^INDIAN^99TRA") , "race_mod"] = "NATIVE_AMERICAN"

data.loc[(data.race == "Native Hawaiian or Other Pacific Islander") | (data.race == "Native Hawaiian Or Other Pacific Islander") | 
              (data.race == "Native Hawaiian or other Pacific Islander") | (data.race == "Native Hawaiian or Pacific Islander") | 
              (data.race == "Other Pacific Islander") | (data.race == "Native Hawaiian or Pacific ISL") | 
              (data.race == "HAWAIIAN") | (data.race == "PAC IS/HI"), "race_mod"] = "PACIFIC_ISLANDER"

data.loc[(data.race == "HISPANIC") | (data.race == "H^HISPANIC^99TRA"), "race_mod"] = "HISPANIC"

data.loc[(data.race == "Middle Eastern or North African") | (data.race == "8") | (data.race == "D") | (data.race == "A") | 
         (data.race == "U~O") | (data.race == "I~O") | (data.race == "6~3") | (data.race == "3~1") | (data.race == "I~B") | 
         (data.race == "1~O") | (data.race == "Arab"), "race_mod"] = "UNDETERMINED"

In [18]:
data.race_mod.unique()

array(['UNKNOWN', 'ASIAN', 'WHITE', 'NATIVE_AMERICAN', 'BLACK', 'OTHER',
       'PACIFIC_ISLANDER', 'UNDETERMINED', 'HISPANIC'], dtype=object)

<font size=7>Ethnicity

In [19]:
data.ethnicity.nunique()

20

In [20]:
data["ethnicity_mod"] = np.nan

In [21]:
data.loc[(data.ethnicity == "Hispanic Or Latino") | (data.ethnicity == "Hispanic or Latino") | 
         (data.ethnicity == "Hispanic/Latino") | (data.ethnicity == "Hispanic") | 
         (data.ethnicity == "HISPANIC") | (data.ethnicity == "hispanic"), "ethnicity_mod"] = "HISPANIC"

data.loc[(data.ethnicity == "Not Hispanic or Latino") | (data.ethnicity == "Not Hispanic Or Latino") | 
         (data.ethnicity == "NON-HISPANIC") | (data.ethnicity == "Non-Hispanic") | 
         (data.ethnicity == "non hispanic"), "ethnicity_mod"] = "NOT_HISPANIC"

data.loc[(data.ethnicity == "Unknown") | (data.ethnicity == "Patient Refused") | 
         (data.ethnicity == "PT DECLINED OR UNAVAILABLE") | (data.ethnicity == "Declined to Specify") | 
         (data.ethnicity == "ENGLISH") | (data.ethnicity == "unknown") | (data.ethnicity == "WHITE") | 
         (data.ethnicity == "UNKNOWN") | (data.ethnicity == "U") | data.ethnicity.isnull(), "ethnicity_mod"] = "UNKNOWN"

In [22]:
data.ethnicity_mod.unique()

array(['UNKNOWN', 'NOT_HISPANIC', 'HISPANIC'], dtype=object)

<font size=7>Language

In [23]:
data.language.nunique()

88

In [24]:
data["language_mod"] = np.nan

In [25]:
#1
data.loc[(data.language == "en") | (data.language == "eng") | 
         (data.language == "EN") | (data.language == "ENGLISH") | 
         (data.language == "ENG"), "language_mod"] = "ENGLISH"
#2
data.loc[(data.language == "es") | (data.language == "SP") | 
         (data.language == "spa") | (data.language == "SPANISH"), "language_mod"] = "SPANISH"
#3
data.loc[(data.language == "UNK") | (data.language == "UN") | (data.language == "und") | 
         (data.language.isnull()), "language_mod"] = "UNKNOWN"
#4
data.loc[(data.language == "fr") | (data.language == "FR"), "language_mod"] = "FRENCH"
#5
data.loc[(data.language == "br") | (data.language == "BR") | (data.language == "pt") | 
         (data.language == "pr") | (data.language == "PR"), "language_mod"] = "PORTUGUESE"
#6
data.loc[(data.language == "ru") | (data.language == "RU"), "language_mod"] = "RUSSIAN"
#7
data.loc[(data.language == "ko") | (data.language == "KO"), "language_mod"] = "KOREAN"
#8
data.loc[(data.language == "ja") | (data.language == "JA"), "language_mod"] = "JAPANESE"
#9
data.loc[(data.language == "it") | (data.language == "IT"), "language_mod"] = "ITALIAN"
#10
data.loc[(data.language == "sw") | (data.language == "SW"), "language_mod"] = "SWAHILI"
#11
data.loc[(data.language == "de") | (data.language == "GE"), "language_mod"] = "GERMAN"
#12
data.loc[(data.language == "he") | (data.language == "HE") | (data.language == "yi"), "language_mod"] = "HEBREW_YIDDISH"
#13
data.loc[(data.language == "th") | (data.language == "TH"), "language_mod"] = "THAI"
#14
data.loc[(data.language == "ur") | (data.language == "UR"), "language_mod"] = "URDU"
#15
data.loc[(data.language == "gu") | (data.language == "GU") | (data.language == "HI") | (data.language == "hi") | 
         (data.language == "pa"), "language_mod"] = "HINDI_GUJARATI_PUNJABI"
#16
data.loc[(data.language == "SO") | (data.language == "so"), "language_mod"] = "SOMALI"
#17
data.loc[(data.language == "ar") | (data.language == "AR"), "language_mod"] = "ARABIC"
#18
data.loc[(data.language == "zh"), "language_mod"] = "CHINESE"
#19
data.loc[(data.language == "bg"), "language_mod"] = "BULGARIAN"
#20
data.loc[(data.language == "VI") | (data.language == "vi"), "language_mod"] = "VIETNAMESE"
#21
data.loc[(data.language == "fa"), "language_mod"] = "PERSIAN"
#22
data.loc[(data.language == "tr") | (data.language == "ug"), "language_mod"] = "TURKISH_UIGHUR"
#23
data.loc[(data.language == "uk"), "language_mod"] = "UKRANIAN"
#24
data.loc[(data.language == "af"), "language_mod"] = "AFRIKAANS"
#25
data.loc[(data.language == "km"), "language_mod"] = "KHMER"
#26
data.loc[(data.language == "ie"), "language_mod"] = "INTERLINGUE"
#27
data.loc[(data.language == "et"), "language_mod"] = "ESTONIAN"
#28
data.loc[(data.language == "el") | (data.language == "grc"), "language_mod"] = "GREEK"
#29
data.loc[(data.language == "ht"), "language_mod"] = "HAITIAN"
#30
data.loc[(data.language == "TA"), "language_mod"] = "TAMIL"
#31
data.loc[(data.language == "pl"), "language_mod"] = "POLISH"
#32
data.loc[(data.language == "dv"), "language_mod"] = "DIHEVI"
#33
data.loc[(data.language == "cs"), "language_mod"] = "CZECH"
#34
data.loc[(data.language == "nl"), "language_mod"] = "DUTCH"
#35
data.loc[(data.language == "lo"), "language_mod"] = "LAO"
#36
data.loc[(data.language == "sr"), "language_mod"] = "SERBIAN"
#37
data.loc[(data.language == "ro"), "language_mod"] = "ROMANIAN"
#38
data.loc[(data.language == "tl"), "language_mod"] = "TAGALOG"
#39
data.loc[(data.language == "id"), "language_mod"] = "INDONESIAN"
#40
data.loc[(data.language == "da"), "language_mod"] = "DANISH"
#41
data.loc[(data.language == "ee"), "language_mod"] = "EWE"
#42
data.loc[(data.language == "eo"), "language_mod"] = "ESPERANTO"
#43
data.loc[(data.language == "fo"), "language_mod"] = "FAROESE"
#44
data.loc[(data.language == "sg"), "language_mod"] = "SANGO"
#45
data.loc[(data.language == "sv"), "language_mod"] = "SWEDISH"
#46
data.loc[(data.language == "ne"), "language_mod"] = "NEPALI"
#47
data.loc[(data.language == "AOF") | (data.language == "OT") | (data.language == "E") | (data.language == "phi") | 
         (data.language == "CI") | (data.language == "PO") | (data.language == "HM") | (data.language == "sgn") | 
         (data.language == "nai") | (data.language == "haw"), "language_mod"] = "UNDETERMINED"

In [26]:
data.language_mod.unique()

array(['ENGLISH', 'UNKNOWN', 'SPANISH', 'UNDETERMINED', 'INTERLINGUE',
       'AFRIKAANS', 'CHINESE', 'ITALIAN', 'PORTUGUESE', 'ARABIC',
       'FRENCH', 'ESTONIAN', 'VIETNAMESE', 'BULGARIAN', 'KHMER',
       'HAITIAN', 'JAPANESE', 'GREEK', 'KOREAN', 'RUSSIAN', 'THAI',
       'HINDI_GUJARATI_PUNJABI', 'URDU', 'SOMALI', 'TAMIL', 'SWAHILI',
       'GERMAN', 'HEBREW_YIDDISH', 'DIHEVI', 'POLISH', 'CZECH', 'DUTCH',
       'TURKISH_UIGHUR', 'PERSIAN', 'LAO', 'SERBIAN', 'ROMANIAN',
       'TAGALOG', 'INDONESIAN', 'UKRANIAN', 'DANISH', 'FAROESE', 'EWE',
       'ESPERANTO', 'SANGO', 'SWEDISH', 'NEPALI'], dtype=object)

<font size=7>Encounter Class

In [27]:
data.encounter_class.nunique()

95

In [28]:
data["encounter_class_mod"] = np.nan

In [29]:
data.loc[(data.encounter_class == "EMERGENCY") | (data.encounter_class == "Emergency") |
         (data.encounter_class == "EMERGENCY ROOM") | (data.encounter_class == "EMS") | 
         (data.encounter_class == "EMS REFUSL") | (data.encounter_class == "EMERGENCY ROOM/OBSERVATION") | 
         (data.encounter_class == "PREADMIT") | (data.encounter_class == "FAST TRACK"), "encounter_class_mod"] = "EMERGENCY"

data.loc[(data.encounter_class == "INPATIENT") | (data.encounter_class == "Inpatient") | 
         (data.encounter_class == "SWING INPATIENT") | (data.encounter_class == "I")  | 
         (data.encounter_class == "Swing Inpatient"), "encounter_class_mod"] = "INPATIENT"

data.loc[(data.encounter_class == "OUTPATIENT") | (data.encounter_class == "OUTPT HOSP") |
         (data.encounter_class == "HOSP OUTPT SURGERY") | (data.encounter_class == "Outpatient Diagnostic") |
         (data.encounter_class == "Hospital Outpatient Surgery") | (data.encounter_class == "Outpatient/Ambulatory") | 
         (data.encounter_class == "OPSU") | (data.encounter_class == "Outpatient") | 
         (data.encounter_class == "OUTPTSRVS") | (data.encounter_class == "O") | 
         (data.encounter_class == "BACKLD/OUT"), "encounter_class_mod"] = "OUTPATIENT"

data.loc[(data.encounter_class == "CANCER CTR") | (data.encounter_class == "Oncology Series 30 Days") | 
         (data.encounter_class == "THERAPIES SERIES") | (data.encounter_class == "Therapies Series") |
         (data.encounter_class == "INFUSION SERIES") | (data.encounter_class == "Infusion Series") |
         (data.encounter_class == "Radiation Onc Series 90 Days") | (data.encounter_class == "Outpatient Series")  | 
         (data.encounter_class == "REHAB OP"), "encounter_class_mod"] = "CANCER_CARE"

data.loc[(data.encounter_class == "EXTENDED RECOVERY") | (data.encounter_class == "Extended Recovery") | 
         (data.encounter_class == "EXT RECVRY"), "encounter_class_mod"] = "EXTENDED_RECOVERY"

data.loc[(data.encounter_class == "SNF Inpatient") | (data.encounter_class == "SNF INPATIENT") | 
         (data.encounter_class == "SNF") | (data.encounter_class == "NURSING HOME"), 
         "encounter_class_mod"] = "NURSING_FACILITY"

data.loc[(data.encounter_class == "OBSERVATIO") | (data.encounter_class == "Observation") | 
         (data.encounter_class == "OBSVR. PAT"), "encounter_class_mod"] = "OBSERVATION"

data.loc[(data.encounter_class == "NEWBORN") | (data.encounter_class == "Newborn") | (data.encounter_class == "NICU/TN NB") | 
         (data.encounter_class == "NICU"), "encounter_class_mod"] = "NEONATAL"

data.loc[(data.encounter_class == "BORDER PSYCH") | (data.encounter_class == "Boarder Psychiatric") | 
         (data.encounter_class == "PSYCHIATRY") | (data.encounter_class == "BEH. SC.") |
         (data.encounter_class == "BP"), "encounter_class_mod"] = "PSYCH"

data.loc[(data.encounter_class == "Surgery Admit") | (data.encounter_class == "SURGERY ADMIT") | 
         (data.encounter_class == "AMBSURGERY") | (data.encounter_class == "SA") |
         (data.encounter_class == "SDS") | (data.encounter_class == "ASU"), "encounter_class_mod"] = "SURGERY"

data.loc[(data.encounter_class == "Extended Diagnostic") | (data.encounter_class == "EXTENDED DIAGNOSTIC") | 
         (data.encounter_class == "IMAGE CNTR"), "encounter_class_mod"] = "EXTENDED_DIAGNOSTIC"

data.loc[(data.encounter_class == "HOME HEALTH") | (data.encounter_class == "Home Health"), 
         "encounter_class_mod"] = "HOME_HEALTH"

data.loc[(data.encounter_class == "SPECIMEN") | (data.encounter_class == "Specimen") |  (data.encounter_class == "LABORATORY"),
         "encounter_class_mod"] = "SPECIMEN"

data.loc[(data.encounter_class == "HOSPICE") | (data.encounter_class == "Hospice") | (data.encounter_class == "HOS"),
         "encounter_class_mod"] = "HOSPICE"

data.loc[(data.encounter_class == "HEALTHY FOR LIVING") | (data.encounter_class == "WELLNESS") | 
         (data.encounter_class == "OCCUPHEALT"), "encounter_class_mod"] = "GENERAL_WELLNESS"

data.loc[(data.encounter_class == "CAP") | (data.encounter_class == "CAP/Medicaid") | 
         (data.encounter_class == "CAP MCAID"), "encounter_class_mod"] = "COMMUNITY_ALTERNATIVES_PROGRAM"

data.loc[(data.encounter_class == "DOSHER MEDICAL ASSOCIATES") | (data.encounter_class == "ST PAULS") | 
         (data.encounter_class == "URGENT CARE SE MEDICAL MALL"), 
         "encounter_class_mod"] = "FACILITIES"

data.loc[(data.encounter_class == "CATH/EP PROC") | (data.encounter_class == "Cath/EP Procedures"), 
         "encounter_class_mod"] = "CATH_EP"

data.loc[(data.encounter_class == "PRIV PLUS") | (data.encounter_class == "FMONT MED") | 
         (data.encounter_class == "WH PROF SV") | (data.encounter_class == "MINOR EMR") | 
         (data.encounter_class == "SRMCFP"), "encounter_class_mod"] = "SRMC_NETWORK"

data.loc[(data.encounter_class == "DECEASED ORGAN DONOR") | (data.encounter_class == "DIABETIC") | 
         (data.encounter_class == "RECURRING"), "encounter_class_mod"] = "OTHER"

data.loc[(data.encounter_class == "NOT GIVEN"), "encounter_class_mod"] = "UNKNOWN"

data.loc[(data.encounter_class == "1") | (data.encounter_class == "CL") | (data.encounter_class == "R") | 
         (data.encounter_class == "RO") | (data.encounter_class == "A") | (data.encounter_class == "EV"), 
         "encounter_class_mod"] = "UNDETERMINED"

In [30]:
data.encounter_class_mod.unique()

array(['OUTPATIENT', 'EMERGENCY', 'UNKNOWN', 'INPATIENT',
       'EXTENDED_DIAGNOSTIC', 'SPECIMEN', 'EXTENDED_RECOVERY', 'SURGERY',
       'CANCER_CARE', 'OBSERVATION', 'CATH_EP', 'PSYCH', 'OTHER',
       'SRMC_NETWORK', 'HOSPICE', 'NEONATAL', 'NURSING_FACILITY',
       'UNDETERMINED', 'COMMUNITY_ALTERNATIVES_PROGRAM', 'FACILITIES',
       'HOME_HEALTH', 'GENERAL_WELLNESS'], dtype=object)

<font size=7>Admit Type

In [31]:
data.admit_type.unique()

array([nan, 'Emergency', 'Elective', 'Outpatient', 'Inpatient',
       'EMERGENCY', 'ELECTIVE', 'DR. REFER', '3', 'Urgent', '1',
       'DR REFER', '2', 'Information Not Available', 'URGENT', 'Newborn',
       'NEWBORN', 'TR', 'IN', 'COMMITTED', 'Y', 'Unknown', 'TRAUMA', '4',
       'SNF/ICF', '9', 'ACUTE HOSP', 'HMO', 'READMISSIO', '5'],
      dtype=object)

In [32]:
data["admit_type_mod"] = np.nan

In [33]:
data.loc[(data.admit_type == "Emergency") | (data.admit_type == "EMERGENCY") | (data.admit_type == "URGENT") | 
         (data.admit_type == "Urgent") | (data.admit_type == "ACUTE HOSP") | 
         (data.admit_type == "TRAUMA") | (data.admit_type == "TR"), "admit_type_mod"] = "EMERGENCY"

data.loc[(data.admit_type == "Elective") | (data.admit_type == "ELECTIVE"), "admit_type_mod"] = "ELECTIVE"

data.loc[(data.admit_type == "Newborn") | (data.admit_type == "NEWBORN"), "admit_type_mod"] = "NEWBORN"

data.loc[(data.admit_type == "DR. REFER") | (data.admit_type == "DR REFER") | 
         (data.admit_type == "HMO"), "admit_type_mod"] = "DOCTOR_REFERAL"

data.loc[(data.admit_type == "1") | (data.admit_type == "2") | (data.admit_type == "3") | (data.admit_type == "4") | 
         (data.admit_type == "5"), "admit_type_mod"] = "TRIAGE"

data.loc[(data.admit_type == "Inpatient") | (data.admit_type == "IN") | (data.admit_type == "COMMITTED") | 
         (data.admit_type == "READMISSIO"),  "admit_type_mod"] = "INPATIENT"

data.loc[(data.admit_type == "Outpatient"), "admit_type_mod"] = "OUTPATIENT"

data.loc[(data.admit_type == "SNF/ICF"), "admit_type_mod"] = "SNF_ICF"

data.loc[(data.admit_type == "Information Not Available") | (data.admit_type == "Unknown") | 
         (data.admit_type.isnull()), "admit_type_mod"] = "UNKNOWN"

data.loc[(data.admit_type == "Y") | (data.admit_type == "9"), "admit_type_mod"] = "UNDETERMINED"

In [34]:
data.admit_type_mod.unique()

array(['UNKNOWN', 'EMERGENCY', 'ELECTIVE', 'OUTPATIENT', 'INPATIENT',
       'DOCTOR_REFERAL', 'TRIAGE', 'NEWBORN', 'UNDETERMINED', 'SNF_ICF'],
      dtype=object)

<font size=7>Admit Source

In [35]:
data.admit_source.unique()

array([nan, 'Non-Health Care Facility Point of*', 'PHYSICIAN REFERRAL',
       'HOME/PHYS OFF/WORKPLACE', 'Not available',
       'Transfer from a hospital (different facility)', 'Clinic Referral',
       'Information not available', 'Transfer from Hospital (Different*',
       'WALK-IN', '12', 'Q', 'Information Unavailable',
       'Physician Referral', 'S', 'TRANSFER FROM SNF', 'EMERGENCY ROOM',
       'Born Inside this Hospital', 'A Ambulatory Surgery Center',
       'NORMAL DELIVERY', 'Transfer from SNF or Intermediate*',
       'DIRECT PHYS OFF/OTHER FAC', 'Emergency Room',
       'Psych, Substance Abuse, or Rehab*',
       'Transfer from Skilled Nursing Facility', 'CHE',
       'COURT/LAW ENFORCEMENT', 'Court/Law Enforcement', '9 Hospice',
       'Transfer from another healthcare facility',
       'Transfer from a Skilled Nursing Facility', '103',
       '6 - Transfer from Another Health Center', 'Premature Delivery',
       '4 Born Outside this Hospital', 'OTHER RESCUE DIRECT AD

In [36]:
data["admit_source_mod"] = np.nan

In [37]:
data.loc[(data.admit_source == "PHYSICIAN REFERRAL") | 
         (data.admit_source == "Clinic Referral") | 
         (data.admit_source == "Physician Referral") | 
         (data.admit_source == "2 - Clinic Referral") | 
         (data.admit_source == "3 - HMO Referral") | 
         (data.admit_source == "HMO Referral") | 
         (data.admit_source == "Non-Health Care Facility Point of*"), "admit_source_mod"] = "REFERRAL"

data.loc[(data.admit_source == "Transfer from a hospital (different facility)") | 
         (data.admit_source == "Transfer from Hospital (Different*") | 
         (data.admit_source == "TRANSFER FROM SNF") | 
         (data.admit_source == "Transfer from SNF or Intermediate*") | 
         (data.admit_source == "Transfer from Skilled Nursing Facility") | 
         (data.admit_source == "Transfer from another healthcare facility") | 
         (data.admit_source == "Transfer from a Skilled Nursing Facility") | 
         (data.admit_source == "Transfer from A Skilled Nursing Facility") | 
         (data.admit_source == "Transfer From A Skilled Nursing Facility") | 
         (data.admit_source == "6 - Transfer from Another Health Center") | 
         (data.admit_source == "TRAN FROM HOSP/EXTRAMURAL") | 
         (data.admit_source == "TRANSFER FRM ACUTE CARE HOSP") | 
         (data.admit_source == "TRANSFER NOT ACUTE/SNF") | 
         (data.admit_source == "Transfer From A Hospital") | 
         (data.admit_source == "Transfer from a hospital") | 
         (data.admit_source == "TR")| 
         (data.admit_source == "TRF FRM HOSPICE&UNDER PLA") |
         (data.admit_source == "9 Hospice"), "admit_source_mod"] = "TRANSFER"

data.loc[(data.admit_source == "Born Inside this Hospital") | 
         (data.admit_source == "Premature Delivery") | 
         (data.admit_source == "NORMAL DELIVERY") |
         (data.admit_source == "4 Born Outside this Hospital") | 
         (data.admit_source == "BABY BORN INSIDE THE HOSP") | 
         (data.admit_source == "BABY BORN OUTSIDE THE HSP") | 
         (data.admit_source == "NORMAL NEWBORN") | 
         (data.admit_source == "SICK BABY") | 
         (data.admit_source == "Sick Baby"), "admit_source_mod"] = "NEONATAL_PEDIATRIC"

data.loc[(data.admit_source == "COURT/LAW ENFORCEMENT") | 
         (data.admit_source == "Court/Law Enforcement") | 
         (data.admit_source == "LAW ENFOURCEMENT"), "admit_source_mod"] = "LAW_COURT"

data.loc[(data.admit_source == "EMERGENCY ROOM") | 
         (data.admit_source == "A Ambulatory Surgery Center") | 
         (data.admit_source == "Emergency Room"), "admit_source_mod"] = "EMERGENCY"

data.loc[(data.admit_source == "Psych, Substance Abuse, or Rehab*") | 
         (data.admit_source == "CHE"), "admit_source_mod"] = "PSYCH"

data.loc[(data.admit_source == "WALK-IN") | 
         (data.admit_source == "HOME/PHYS OFF/WORKPLACE"), "admit_source_mod"] = "WALK_IN"

data.loc[(data.admit_source == "DIRECT PHYS OFF/OTHER FAC") | 
         (data.admit_source == "OTHER RESCUE DIRECT ADMIT"), "admit_source_mod"] = "DIRECT_ADMIT"

data.loc[(data.admit_source == "PRIVILEDGE PLUS +"), "admit_source_mod"] = "INSURANCE"

data.loc[(data.admit_source == "Not available") | (data.admit_source == "Information not available") | 
         (data.admit_source == "Information Unavailable") | data.admit_source.isnull() | 
         (data.admit_source == "INFO NOT AVAILABLE"), "admit_source_mod"] = "UNKNOWN"

data.loc[(data.admit_source == "12") | 
         (data.admit_source == "Q") | 
         (data.admit_source == "S") | 
         (data.admit_source == "103") | 
         (data.admit_source == "J") | 
         (data.admit_source == "100") | 
         (data.admit_source == "11") | 
         (data.admit_source == "106") | 
         (data.admit_source == "10") | 
         (data.admit_source == "104") | 
         (data.admit_source == "101") | 
         (data.admit_source == "102") | 
         (data.admit_source == "SF") | 
         (data.admit_source == "O"), "admit_source_mod"] = "UNDETERMINED"

In [38]:
data.admit_source_mod.unique()

array(['UNKNOWN', 'REFERRAL', 'WALK_IN', 'TRANSFER', 'UNDETERMINED',
       'EMERGENCY', 'NEONATAL_PEDIATRIC', 'DIRECT_ADMIT', 'PSYCH',
       'LAW_COURT', 'INSURANCE'], dtype=object)

<font size=7>Disposition

In [39]:
data.disposition.nunique()

119

In [40]:
data["disposition_mod"] = np.nan

In [41]:
data.loc[(data.disposition == "01 Discharged to home or self care") |
         (data.disposition == "06 Discharged / Transferred to home under care of organized home health service organization") |  
         (data.disposition == "HOM") |  
         (data.disposition == "HOMEANF") | 
         (data.disposition == "HOMESNF") | 
         (data.disposition == "HOMEDAVITA") | 
         (data.disposition == "1") | 
         (data.disposition == "6") |  
         (data.disposition == "81"), "disposition_mod"] = "HOME"

data.loc[(data.disposition == "20 Expired") | 
         (data.disposition == "20 Expired (or did not recover - Christian Science Patient)") |  
         (data.disposition == "20 Expired at home (hospice claims only)") |  
         (data.disposition == "20"), "disposition_mod"] = "EXPIRED"

data.loc[(data.disposition == "03 Discharged / Transferred to SNF") | 
         (data.disposition == "3") |
         (data.disposition == "04 Discharged / Transferred to an ICF") | 
         (data.disposition == "SNF") | 
         (data.disposition == "ICF") | 
         (data.disposition == "4") | 
         (data.disposition == "04 Discharged / Transferred to an ICF") |  
         (data.disposition == "61") |
         (data.disposition == "83") | 
         (data.disposition == "84") | 
         (data.disposition == "89") | 
         (data.disposition == "92"), "disposition_mod"] = "SNF_ICF"

data.loc[(data.disposition == "DMH") | 
         (data.disposition == "DUK") |  
         (data.disposition == "DUKE"), "disposition_mod"] = "DUKE"

data.loc[(data.disposition == "CAR") | 
         (data.disposition == "CHA") |  
         (data.disposition == "CAROLINA") |  
         (data.disposition == "CHAPELHILL"), "disposition_mod"] = "CHAPEL_HILL"

data.loc[(data.disposition == "NHRMC") | 
         (data.disposition == "NHR"), "disposition_mod"] = "NHRMC"

data.loc[(data.disposition == "2") | 
         (data.disposition == "02 Discharged / Transferred to Hospital") | 
         (data.disposition == "9") |
         (data.disposition == "63") | 
         (data.disposition == "63 Discharged/Transferred to long term care hospital") |
         (data.disposition == "66 Discharged/transferred to a Critical Access Hospital (CAH)") | 
         (data.disposition == "82") | 
         (data.disposition == "94"), "disposition_mod"] = "HOSPITAL"

data.loc[(data.disposition == "65 Discharged/Transferred to a Psychiatric Hospital") | 
         (data.disposition == "65") | 
         (data.disposition == "93"), "disposition_mod"] = "PSYCH"

data.loc[(data.disposition == "62") | 
         (data.disposition == "90"), "disposition_mod"] = "REHAB_IRF"

data.loc[(data.disposition == "07 Left against medical advice, or discontinued care") | 
         (data.disposition == "7") | 
         (data.disposition == "AMA") | 
         (data.disposition == "LWT") | 
         (data.disposition == "LWBS"), "disposition_mod"] = "LEFT_AGAINST_MED_ADVICE"

data.loc[(data.disposition == "5") | 
         (data.disposition == "05 Discharged / Transferred to another type of institution") |  
         (data.disposition == "OTH") |  
         (data.disposition == "70") |  
         (data.disposition == "85"), "disposition_mod"] = "MISC_FACILITY"

data.loc[(data.disposition == "50") | 
         (data.disposition == "50 Hospice - home") | 
         (data.disposition == "51 Hospice - medical facility") | 
         (data.disposition == "51"), "disposition_mod"] = "HOSPICE"

data.loc[(data.disposition == "43") | 
         (data.disposition == "NAVAL") | 
         (data.disposition == "VA") | 
         (data.disposition == "NAV"), "disposition_mod"] = "MILITARY"

data.loc[(data.disposition == "HHS") | 
         (data.disposition == "HHSASSIST"), "disposition_mod"] = "HEALTH_HUMAN_SERVICES"

data.loc[(data.disposition == "VID") | 
         (data.disposition == "VIDANT"), "disposition_mod"] = "VIDANT"

data.loc[(data.disposition == "87"), "disposition_mod"] = "COURT_LAW"

data.loc[(data.disposition == "8") |
         (data.disposition == "08") | 
         (data.disposition == "80") | 
         (data.disposition == "96") | 
         (data.disposition == "98") | 
         (data.disposition == "99"), "disposition_mod"] = "RESERVED_NATIONAL_ASSIGNMENT"

data.loc[(data.disposition == "30"), "disposition_mod"] = "STILL_PATIENT"

onslow = ["LWB","LWS","XPH","XIR","AOB","ELO","AIP","XLA","EGWOA","STH","EGW","LTC","HOSM","HOS","AOBS","LABORCHECK","LAB",
          "TERTIARY","XFH","ELWOA","XLAW","XNF","XCAN"]
data.loc[(data.disposition.isin(onslow)), "disposition_mod"] = "ONSLOW_MEMORIAL_HOS_CODES"

alphabetical_codes = ["A","B","C","D","E","F","G","H","I","J","L","N","O","P","R","S","X"]
data.loc[(data.disposition.isin(alphabetical_codes)), "disposition_mod"] = "UNKNOWN_ALPHABETICAL_CODES"

data.loc[(data.disposition.isnull()), "disposition_mod"] = "UNKNOWN"

data.loc[(data.disposition == "0") |
         (data.disposition == "AOF") | 
         (data.disposition == "CD") | 
         (data.disposition == "LWD"), "disposition_mod"] = "UNDETERMINED"

In [42]:
data.disposition_mod.unique()

array(['UNKNOWN', 'UNKNOWN_ALPHABETICAL_CODES', 'UNDETERMINED', 'EXPIRED',
       'HOME', 'MISC_FACILITY', 'LEFT_AGAINST_MED_ADVICE',
       'ONSLOW_MEMORIAL_HOS_CODES', 'SNF_ICF', 'HOSPITAL', 'REHAB_IRF',
       'CHAPEL_HILL', 'DUKE', 'VIDANT', 'HEALTH_HUMAN_SERVICES', 'NHRMC',
       'HOSPICE', 'MILITARY', 'RESERVED_NATIONAL_ASSIGNMENT', 'PSYCH',
       'STILL_PATIENT', 'COURT_LAW'], dtype=object)

<font size=7>Financial Class

In [43]:
data.financial_class.nunique()

190

In [44]:
data["financial_class_mod"] = np.nan
test_insurances =  ["TPL OP","SBE","GBR","BBD","MBX","MBP","GBV","CBY","CBI","MBK","XBN","XBU","PNH","DBC","XBV","POT",
                    "XBO","GBT","MBV","BBG","GBN","MBO","DBD","GBM","GBH","GBS","GHI OP","GBU","XPV","CBT","CBS","GBX",
                    "XBS","CBP","CBU","XBM","CBG","MBM","MPV","GB1","TPL THERAPY"]
dosher_insurance = data[(data.financial_class.isin(test_insurances)) & 
                        (data.facility == "Dosher Memorial Hospital")].financial_class.unique()
data[data.financial_class.isin(dosher_insurance)].facility.unique()

array(['Dosher Memorial Hospital'], dtype=object)

In [45]:
dosher = ["C","C02","C09","C10","C17","C18","C20","C21","C25","C26","C32","C33","C34","C37","C38","C46","C50","C52","C54",
          "C55","CB","CB1","CB2","CB4","CB6","CB8","CB9","CT","CW","GL","MF","PL","CBG"]
data[data.financial_class.isin(dosher)].facility.unique()

array(['Dosher Memorial Hospital'], dtype=object)

In [46]:
srmc = ["D","F","T","V","W"]
data[data.financial_class.isin(srmc)].facility.unique()

array(['Southeastern Regional Medical Center'], dtype=object)

In [47]:
data.loc[(data.financial_class == "Unknown") | 
         (data.financial_class == "U") | 
         (data.financial_class.isnull()), "financial_class_mod"] = "UNKNOWN"

bcbs_prefixes = ["CBY","CBI","CVA","CBT","CBS","CBP","CBU","BBG","DBC","TCE","SBE","GBR","GBV","GBT","GBN","GBM","GBH","GBS",
                 "GBU","GBX","MBK","MBM","MBO","MBP","MBV","MBX","MPV","PNH","POT","XBM","XBN","XBO","XBS","XBU","XPV"]
data.loc[(data.financial_class == "Blue Cross") | 
         (data.financial_class == "BCBS") | 
         (data.financial_class == "Blue Cross-Out of State") | 
         (data.financial_class.isin(bcbs_prefixes)), "financial_class_mod"] = "BLUE_CROSS"

data.loc[(data.financial_class == "SELF") | 
         (data.financial_class == "Self-pay") | 
         (data.financial_class == "Self-Pay") | 
         (data.financial_class == "PRIVATE PAY") | 
         (data.financial_class == "PRIVATE PAY THERAPY"), "financial_class_mod"] = "SELF_PAY"

data.loc[(data.financial_class == "Veteran's Affairs") | 
         (data.financial_class == "Veterans Affairs") | 
         (data.financial_class == "Veterans' Affairs"), "financial_class_mod"] = "VETERANS_AFFAIRS"

data.loc[(data.financial_class == "Medicare") | 
         (data.financial_class == "Medicare Health Maintenance Organization (HMO)") | 
         (data.financial_class == "MCRE ADV") | 
         (data.financial_class == "CARE") | 
         (data.financial_class == "HUMANA MEDICARE OP") | 
         (data.financial_class == "Medicare HMO") | 
         (data.financial_class == "UNITED HEALTHCARE MEDICARE") | 
         (data.financial_class == "BLUE MEDICARE OP") | 
         (data.financial_class == "AETNA MEDICARE OPEN OP") | 
         (data.financial_class == "BLUE MEDICARE IP") | 
         (data.financial_class == "UNITED HEALTHCARE MEDICARE I/P") | 
         (data.financial_class == "HUMANA MEDICARE IP") | 
         (data.financial_class == "Medicare Health Maintenance Organization") | 
         (data.financial_class == "UNICARE MEDICARE IP") | 
         (data.financial_class == "CIGNA MEDICARE ACCESS") | 
         (data.financial_class == "UNICARE MEDICARE OP") | 
         (data.financial_class == "MVP CARE/USA CARE MEDICARE OP") | 
         (data.financial_class == "MVP CARE/USA CARE MEDICARE IP") | 
         (data.financial_class == "PYRAMID MEDICARE OP"), "financial_class_mod"] = "MEDICARE"

data.loc[(data.financial_class == "Medicaid") | 
         (data.financial_class == "CAID") | 
         (data.financial_class == "Medicaid Out-of-State") | 
         (data.financial_class == "MCAID SC") | 
         (data.financial_class == "MCD MGD CARE") | 
         (data.financial_class == "Institutional"), "financial_class_mod"] = "MEDICAID"

data.loc[(data.financial_class == "MEDICARE & MEDICAID"), "financial_class_mod"] = "MEDICARE_MEDICAID"

data.loc[(data.financial_class == "Workers' Compensation") | 
         (data.financial_class == "WC") | 
         (data.financial_class == "Worker's Compensation"), "financial_class_mod"] = "WORKERS_COMP"

data.loc[(data.financial_class == "AETNA US HEALTHCARE OP") | 
         (data.financial_class == "AETNA US HEALTHCARE IP") | 
         (data.financial_class == "AETNA CLINIC"), "financial_class_mod"] = "AETNA"

data.loc[(data.financial_class == "CIGNA OP") | 
         (data.financial_class == "NALC CIGNA") | 
         (data.financial_class == "APWU CIGNA") | 
         (data.financial_class == "CIGNA IP") | 
         (data.financial_class == "CIGNA CLINIC"), "financial_class_mod"] = "CIGNA"

data.loc[(data.financial_class == "MIDWEST LIFE - INS CENTER OP") | 
         (data.financial_class == "MIDWEST LIFE - INS CENTER IP"), "financial_class_mod"] = "MIDWEST_LIFE"

data.loc[(data.financial_class == "UNITED HEALTHCARE IP") | 
         (data.financial_class == "UNITED HEALTHCARE CLINIC") | 
         (data.financial_class == "UNITED HEALTHCARE OP"), "financial_class_mod"] = "UNITED"

data.loc[(data.financial_class == "ASSURANT HEALTH OP") | 
         (data.financial_class == "ASSURANT HEALTH IP"), "financial_class_mod"] = "ASSURANT_HEALTH"

data.loc[(data.financial_class == "AARP OP") | 
         (data.financial_class == "AARP IP"), "financial_class_mod"] = "AARP"

data.loc[(data.financial_class == "GOLDEN RULE IP") | 
         (data.financial_class == "GOLDEN RULE OP"), "financial_class_mod"] = "GOLDEN_RULE"

data.loc[(data.financial_class == "USAA LIFE IP") | 
         (data.financial_class == "Tricare/Champus") | 
         (data.financial_class == "Tricare.Champus"), "financial_class_mod"] = "USAA_TRICARE"

data.loc[(data.financial_class == "MEDCOST OP") | 
         (data.financial_class == "MEDCOST CLINIC") | 
         (data.financial_class == "MEDCOST IP"), "financial_class_mod"] = "MEDCOST"

data.loc[(data.financial_class == "TPL OP") | 
         (data.financial_class == "TPL THERAPY") | 
         (data.financial_class == "Liability") | 
         (data.financial_class == "LIABILITY"), "financial_class_mod"] = "LIABILITY"

data.loc[(data.financial_class == "NC HEALTH CHOICE") | 
         (data.financial_class == "NC HEALTH CHOICE CLINIC"), "financial_class_mod"] = "NC_HEALTH_CHOICE"

data.loc[(data.financial_class == "Commercial") | 
         (data.financial_class == "COMM") | 
         (data.financial_class == "Commercail"), "financial_class_mod"] = "COMMERCIAL"

data.loc[(data.financial_class == "GB1") | 
         (data.financial_class == "HUMANA OP"), "financial_class_mod"] = "HUMANA"

data.loc[(data.financial_class == "CBX") | 
         (data.financial_class == "BBD") | 
         (data.financial_class == "XBV") | 
         (data.financial_class == "DBD"), "financial_class_mod"] = "UNASSIGNED_ALPHA_PREFIX"

data.loc[(data.financial_class.isin(dosher)), "financial_class_mod"] = "DOSHER_MEMORIAL_HOSPITAL"

data.loc[(data.financial_class.isin(srmc)), "financial_class_mod"] = "SRMC"

data.loc[(data.financial_class == "COVENTRY IP") | 
         (data.financial_class == "COVENTRY OP") | 
         (data.financial_class == "WELLPATH OP") | 
         (data.financial_class == "GEHA WELLPATH COVENTRY"), "financial_class_mod"] = "WELLPATH_COVENTRY"

data.loc[(data.financial_class == "HOSP") | 
         (data.financial_class == "Hospice"), "financial_class_mod"] = "HOSPICE"

data.loc[(data.financial_class == "AMERICAS 1ST CHOICE IP") | 
         (data.financial_class == "AMERICAS 1ST CHOICE OP"), "financial_class_mod"] = "AMERICAS_1ST"

data.loc[(data.financial_class == "UNITED MEDICAL RESOURCES IP") | 
         (data.financial_class == "UNITED MEDICAL RESOURCES OP"), "financial_class_mod"] = "UNITED_MEDICAL"

data.loc[(data.financial_class == "State Health Plan"), "financial_class_mod"] = "STATE_HEALTH_PLAN"

data.loc[(data.financial_class == "Preferred Provider Organization (PPO)"), "financial_class_mod"] = "PREFERRED_PROVIDER_ORG"

data.loc[(data.financial_class == "NC BAPTIST ASSEMBLY OP") | 
         (data.financial_class == "POMCO OP") | 
         (data.financial_class == "GHI OP") | 
         (data.financial_class == "MAIL HANDLERS OP") | 
         (data.financial_class == "CORESOURCE OP") | 
         (data.financial_class == "BEMC COOPERATIVE BENEFIT ADM INC OP") | 
         (data.financial_class == "UNICARE OP") | 
         (data.financial_class == "BANKERS LIFE OP") | 
         (data.financial_class == "FAC BILL") | 
         (data.financial_class == "FIN COUNSEL") | 
         (data.financial_class == "SPONS AG") | 
         (data.financial_class == "BRUNS CO HD/BCCCP") | 
         (data.financial_class == "MEDICAL MUTUAL OF OHIO OP") | 
         (data.financial_class == "MUTUAL OF OMAHA OP") | 
         (data.financial_class == "UNITED AMERICAN INS OP") | 
         (data.financial_class == "OT STATE"), "financial_class_mod"] = "MISC"

data.loc[(data.financial_class == "Other") | 
         (data.financial_class == "O") | 
         (data.financial_class == "OTHER"), "financial_class_mod"] = "OTHER"

data.loc[(data.financial_class == "Unknown") | 
         (data.financial_class == "U") | 
         (data.financial_class.isnull()), "financial_class_mod"] = "UNKNOWN"

data.loc[(data.financial_class == "*") | 
         (data.financial_class == "=") | 
         (data.financial_class == "+") | 
         (data.financial_class == "\\F\\"), "financial_class_mod"] = "UNDETERMINED"

In [48]:
data.financial_class_mod.unique()

array(['UNKNOWN', 'BLUE_CROSS', 'OTHER', 'COMMERCIAL', 'MEDICARE',
       'VETERANS_AFFAIRS', 'MEDICAID', 'SELF_PAY', 'USAA_TRICARE',
       'WORKERS_COMP', 'STATE_HEALTH_PLAN', 'HOSPICE', 'LIABILITY',
       'UNITED', 'AARP', 'AETNA', 'DOSHER_MEMORIAL_HOSPITAL', 'SRMC',
       'CIGNA', 'UNASSIGNED_ALPHA_PREFIX', 'HUMANA', 'UNITED_MEDICAL',
       'NC_HEALTH_CHOICE', 'MEDCOST', 'MISC', 'MEDICARE_MEDICAID',
       'PREFERRED_PROVIDER_ORG', 'ASSURANT_HEALTH', 'GOLDEN_RULE',
       'WELLPATH_COVENTRY', 'UNDETERMINED', 'MIDWEST_LIFE',
       'AMERICAS_1ST'], dtype=object)

<font size=7>VIP Indicator

In [49]:
data.vip_indicator.nunique()

6

In [50]:
data["vip_indicator_mod"] = np.nan

In [51]:
data.loc[(data.vip_indicator == "Restricted"), "vip_indicator_mod"] = "RESTRICTED"
data.loc[(data.vip_indicator == "VIP"), "vip_indicator_mod"] = "VIP"
data.loc[(data.vip_indicator == "Normal"), "vip_indicator_mod"] = "NORMAL"
data.loc[(data.vip_indicator == "EMPLOYEE & EMP.  RELATED"), "vip_indicator_mod"] = "EMPLOYEE_&_EMP_RELATED"
data.loc[(data.vip_indicator == "PHYSICIAN & PHY RELATED"), "vip_indicator_mod"] = "PHYSICIAN_&_PHY_RELATED"
data.loc[(data.vip_indicator == "PREFERRED PLUS"), "vip_indicator_mod"] = "PREFERRED_PLUS"
data.loc[(data.vip_indicator.isnull()), "vip_indicator_mod"] = "UNKNOWN"

In [52]:
data.vip_indicator_mod.unique()

array(['UNKNOWN', 'RESTRICTED', 'VIP', 'NORMAL', 'EMPLOYEE_&_EMP_RELATED',
       'PHYSICIAN_&_PHY_RELATED', 'PREFERRED_PLUS'], dtype=object)

<font size=7>Encounter Category

In [53]:
data.encounter_category.nunique()

2

In [54]:
data["encounter_category_mod"] = np.nan

In [55]:
data.loc[(data.encounter_category == "Procedure"), "encounter_category_mod"] = "PROCEDURE"
data.loc[(data.encounter_category == "Diagnosis"), "encounter_category_mod"] = "DIAGNOSIS"
data.loc[(data.encounter_category.isnull()), "encounter_category_mod"] = "UNKNOWN"

In [56]:
data.encounter_category_mod.unique()

array(['DIAGNOSIS', 'PROCEDURE', 'UNKNOWN'], dtype=object)

<font size=7>Encounter Type

In [57]:
data.encounter_type.nunique()

11

In [58]:
data["encounter_type_mod"] = np.nan

In [59]:
data.loc[(data.encounter_type == "ADMIT") | (data.encounter_type == "AD"), "encounter_type_mod"] = "ADMIT"

data.loc[(data.encounter_type == "WORKING") | (data.encounter_type == "Working") | 
         (data.encounter_type == "W"), "encounter_type_mod"] = "WORKING"

data.loc[(data.encounter_type == "FINAL") | (data.encounter_type == "Final") | 
         (data.encounter_type == "F"), "encounter_type_mod"] = "FINAL"

data.loc[(data.encounter_type == "AOF"), "encounter_type_mod"] = "AOF"

data.loc[(data.encounter_type.isnull()), "encounter_type_mod"] = "UNKNOWN"

data.loc[(data.encounter_type == "A") | (data.encounter_type == "S"), "encounter_type_mod"] = "UNDETERMINED"

In [60]:
data.encounter_type_mod.unique()

array(['ADMIT', 'WORKING', 'UNKNOWN', 'AOF', 'FINAL', 'UNDETERMINED'],
      dtype=object)

<font size=7>Encounter Code Method

In [61]:
data.encounter_code_method.nunique()

12

In [62]:
data["encounter_code_method_mod"] = np.nan

In [63]:
data.loc[(data.encounter_code_method == "ICD-9") | (data.encounter_code_method == "ICD9") | 
         (data.encounter_code_method == "I9"), "encounter_code_method_mod"] = "ICD_9"

data.loc[(data.encounter_code_method == "ICD-10") | (data.encounter_code_method == "ICD10") | 
         (data.encounter_code_method == "I10") | (data.encounter_code_method == "I1"), "encounter_code_method_mod"] = "ICD_10"

data.loc[(data.encounter_code_method == "Free Text") | 
         (data.encounter_code_method == "FF"), "encounter_code_method_mod"] = "FREE_TEXT"

data.loc[(data.encounter_code_method == "CPT") | (data.encounter_code_method == "CP"), "encounter_code_method_mod"] = "CPT"

data.loc[(data.encounter_code_method == "H"), "encounter_code_method_mod"] = "HCPCS"

data.loc[(data.encounter_code_method.isnull()), "encounter_code_method_mod"] = "UNKNOWN"

data.loc[(data.encounter_code_method == "C4"), "encounter_code_method_mod"] = "UNDETERMINED"

In [64]:
data.encounter_code_method_mod.unique()

array(['ICD_9', 'FREE_TEXT', 'ICD_10', 'CPT', 'UNKNOWN', 'UNDETERMINED',
       'HCPCS'], dtype=object)

<font size=7>Cleaning Demographic Profiles

In [65]:
# Group all data by patient ID

group = data.groupby("patient_id")

In [66]:
# Only get IDs with multiple occurences in data

data_dup = data.loc[data.patient_id.duplicated(), ["patient_id", "gender", "race", "ethnicity", "language"]]

In [67]:
# All unique, duplicated IDs

uni_ids = data_dup.patient_id.unique()

In [74]:
# Create a list for each demographic feature that would need profile adjustments

gender_fix = []
race_fix = []
eth_fix = []
lang_fix = []

# For each unique ID, get all index locations within dataset

# For each ID
for i, p_id in enumerate(uni_ids):
    j = (i + 1) / len(uni_ids)
    sys.stdout.write('\r')
    
    # Get all index locations for patient ID
    pat_idx = group.groups.get(p_id)
    
    
    # For each demographic below:
    #     Get all indexes of a patient ID
    #     Isolate the demograpic column
    #     Add the patient ID to list if there's more than 1 unique demographic value for the ID
    
    gender_df = data["gender"][pat_idx]
    if gender_df.nunique() > 1:
        gender_fix.append(p_id)
        
    race_df = data["race"][pat_idx]
    if race_df.nunique() > 1:
        race_fix.append(p_id)

    eth_df = data["ethnicity"][pat_idx]    
    if eth_df.nunique() > 1:
        eth_fix.append(p_id)
    
    lang_df = data["language"][pat_idx]
    if lang_df.nunique() > 1:
        lang_fix.append(p_id)
        
        
        
    sys.stdout.write("[%-20s] %d%%" % ('='*int(20*j), 100*j))
    sys.stdout.flush()

In [75]:
# Apparently no profile adjustments are needed, AFTER data cleaning

print("gender: ", len(gender_fix))
print("gender: ", len(race_fix))
print("gender: ", len(eth_fix))
print("gender: ", len(lang_fix))

gender:  0
gender:  0
gender:  0
gender:  0


In [76]:
# Code for cleaning demographic profiles--if in future HIE data queries, this problem presents
# UNKNOWN AND UNDETERMINED are base cases

fix_lists = [gender_fix, race_fix, eth_fix, lang_fix]
fix_columns = ["gender_mod", "race_mod", "ethnicity_mod", "language_mod"]

# For each demographic list/feature
for i, demo_list in enumerate(fix_lists):
    
    # For each ID in the list of inconsistent demographic info
    for p_id in demo_list:
        
        j = (i + 1) / len(demo_list)
        sys.stdout.write('\r')
        
        # Get the indexes of ID to fix
        pat_idx = group.groups.get(p_id)
    
        # Get demographic values of patient
        demo_df = data[fix_columns[i]][pat_idx]
        
        # All unique demographic values of patient
        demo_uni = demo_df.unique().astype(list)
        
        
        
        
        # For IDs with inconsistent demographic info
        if len(demo_uni) > 1:
            
            # Remove UNKNOWN values
            if "UNKNOWN" in demo_uni:
                demo_uni.remove("UNKNOWN")
            
            
            # If there are STILL inconsistent values
            if len(demo_uni) > 1:
                
                # Remove UNDETERMINED values
                if "UNDETERMINED" in demo_uni:
                    demo_uni.remove("UNDETERMINED")
            
            
            
            # If length is 0, then both UNKNOWN and UNDETERMINED were removed--
            # UNDETERMINED carries more value
            if len(demo_uni) == 0:
                data[fix_columns[i]][pat_idx] = "UNDETERMINED"
                
            # If length is 1, it cannot be UNKNOWN, so set all other values to the 1 value left
            elif len(demo_uni) == 1:
                data[fix_columns[i]][pat_idx] = demo_uni[0]
                
            # If multiple inconsistent values after removing UNKNOWNs and UNDETERMINEDs--
            # set patient demographic info to the most frequent value
            else:
                top_demo = data[fix_columns[i]][pat_idx].value_counts()[0]
                data[fix_columns[i]][pat_idx] = top_demo
    
    
    
    
        sys.stdout.write("[%-20s] %d%%" % ('='*int(20*j), 100*j))
        sys.stdout.flush()

In [77]:
# Create csv in local AWS directory

data.to_csv("HIE_cleaned.csv", index=False)
#datato_csv("HIE_cleaned.csv")

In [78]:
# Upload cleaned dataset to AWS S3 bucket

s3.upload_file("HIE_cleaned.csv", Bucket=bucket, Key="HIE_cleaned.csv")